In [ ]:
!mkdir The_end
%cd The_end
!git clone https://github.com/ManishRoy9/MT-Preparation.git

In [ ]:
!pip3 install -r MT-Preparation/requirements.txt

In [ ]:
from google.colab import files

# Upload files
uploaded = files.upload()

Saving Anuvaad.as.txt to Anuvaad.as.txt
Saving Anuvaad.en.txt to Anuvaad.en.txt
Saving filtered_source_texts.txt to filtered_source_texts.txt
Saving filtered_translations.txt to filtered_translations.txt


In [ ]:
!python3 MT-Preparation/filtering/add_tag.py filtered_source_texts.txt bt

Done desubwording! Output: filtered_source_texts.txt.en


In [ ]:
!python3 MT-Preparation/filtering/add_tag.py filtered_translations.txt bt

Done desubwording! Output: filtered_translations.txt.en


In [ ]:
# Specify the file paths
file1_path = 'Anuvaad.en.txt'
file2_path = 'filtered_source_texts.txt.en'
merged_file_path = 'English.txt'

# Read the content of file1
with open(file1_path, 'r') as file1:
    file1_content = file1.read()

# Read the content of file2
with open(file2_path, 'r') as file2:
    file2_content = file2.read()

# Concatenate the contents of file1 and file2
merged_content = file1_content + file2_content

# Write the merged content to the merged file
with open(merged_file_path, 'w') as merged_file:
    merged_file.write(merged_content)

In [ ]:
# Specify the file paths
file1_path = 'Anuvaad.as.txt'
file2_path = 'filtered_translations.txt.en'
merged_file_path = 'Assamese.txt'

# Read the content of file1
with open(file1_path, 'r') as file1:
    file1_content = file1.read()

# Read the content of file2
with open(file2_path, 'r') as file2:
    file2_content = file2.read()

# Concatenate the contents of file1 and file2
merged_content = file1_content + file2_content

# Write the merged content to the merged file
with open(merged_file_path, 'w') as merged_file:
    merged_file.write(merged_content)

In [ ]:
!python3 MT-Preparation/filter.py English.txt Assamese.txt en as

In [ ]:
!python MT-Preparation/1-train_bpe.py English.txt-filtered.en Assamese.txt-filtered.as

In [ ]:
!python3 MT-Preparation/2-subword.py source.model target.model English.txt-filtered.en Assamese.txt-filtered.as

In [ ]:
!python MT-Preparation/train_dev_test_split.py 5500 5500  Assamese.txt-filtered.as.subword English.txt-filtered.en.subword

In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -R /content/The_end/ /content/drive/MyDrive/

In [ ]:
%cd /content/drive/MyDrive/The_end/
!ls

In [ ]:
!pip3 install OpenNMT-py

In [ ]:
 #Create the YAML configuration file
# On a regular machine, you can create it manually or with nano
# Note here we are using some smaller values because the dataset is small
# For larger datasets, consider increasing: train_steps, valid_steps, warmup_steps, save_checkpoint_steps, keep_checkpoint

config = '''# config.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src:   English.txt-filtered.en.subword.train
        path_tgt:   Assamese.txt-filtered.as.subword.train
        transforms: [filtertoolong]
    valid:
        path_src: English.txt-filtered.en.subword.dev
        path_tgt: Assamese.txt-filtered.as.subword.dev
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 43000
tgt_vocab_size: 40000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 150
src_seq_length: 150

# Tokenization options
src_subword_model: source.model
tgt_subword_model: target.model

# Where to save the log file and the output models/checkpoints
log_file: train.log
save_model: models/model.final

# Stop training if it does not imporve after n validations
early_stopping: 7

# Default: 1000 - Save a model checkpoint for each n
save_checkpoint_steps: 1000

# To save space, limit checkpoints to last n
# keep_checkpoint: 3

seed: 3435

# Default: 100000 - Train the model to max n steps
# Increase to 200000 or more for large datasets
# For fine-tuning, add up the required steps to the original steps
train_steps: 10000

# Default: 15000 - Run validation after n steps
valid_steps: 500

# Default: 4000 - for large datasets, try up to 8000
warmup_steps: 1000
report_every: 200

# Number of GPUs, and IDs of GPUs
world_size: 1
gpu_ranks: [0]

# Batching
bucket_size: 262144
num_workers: 0  # Default: 2, set to 0 when RAM out of memory
batch_type: "tokens"
batch_size: 4096   # Tokens per batch, change when CUDA out of memory
valid_batch_size: 2048
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 0.1
# warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 6
dec_layers: 6
heads: 8
hidden_size: 512
word_vec_size: 512
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]
'''

with open("config.yaml", "w+") as config_yaml:
  config_yaml.write(config)

In [ ]:
!cat config.yaml

In [ ]:
!onmt_build_vocab -config config.yaml -n_sample -1 -num_threads 2

In [ ]:
!rm -rf drive/MyDrive/The_end/models/

In [ ]:
!onmt_train -config config.yaml

In [ ]:
!onmt_translate -model models/model.final_step_5500.pt -src  English.txt-filtered.en.subword.test -output Assamese.translated -gpu 0 -min_length 1

In [ ]:
!pip3 install --upgrade -q sentencepiece

In [ ]:
!python3 MT-Preparation/3-desubword.py target.model Assamese.txt-filtered.as.subword.test

In [ ]:
!python3 MT-Preparation/3-desubword.py target.model Assamese.translated

In [ ]:
!wget https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py

--2023-07-23 08:34:14--  https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 957 [text/plain]
Saving to: ‘compute-bleu.py’

compute-bleu.py     100%[===================>]     957  --.-KB/s    in 0s      

2023-07-23 08:34:15 (33.4 MB/s) - ‘compute-bleu.py’ saved [957/957]



In [ ]:
!pip3 install sacrebleu

In [ ]:
!python3 compute-bleu.py Assamese.txt-filtered.as.subword.test.desubword Assamese.translated.desubword